# Seq and PacketBuffer class

## Seq class implementation changed
#### Addition and Subtraction
```Python
seq + n     # forward n steps
seq - n     # backward n steps
seq1 + seq2 # TypeError
```
#### Distance from a Seq object to the other Seq object
```Python
distance(seq1, seq2)  # returns int type
```

In [2]:
from packet import *

base = Seq(253)
nextseqnum = Seq(2)
print('base:', base, 'nextseqnum:', nextseqnum)
print(base.distance(nextseqnum))
print(nextseqnum.distance(base))
print(nextseqnum - 5)

base: Seq(253) nextseqnum: Seq(2)
-5
5
Seq(253)


## PacketBuffer

In [3]:
help(PacketBuffer)

Help on class PacketBuffer in module packet:

class PacketBuffer(builtins.object)
 |  PacketBuffer(gbn, bufsize)
 |  
 |  Packet buffers indexed by Seq bumber for GBNsend's send buffer or GBNrecv's receive buffer
 |  Note: for indexing, gbn.base attribute is used.
 |      popfron method updates gbn.base variable
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, seq)
 |      # support indexing. e.g) sndpkt[Seq(5)]
 |  
 |  __init__(self, gbn, bufsize)
 |      Packet buffer for GBNsend's send buffer or GBNrecv's receive buffer
 |      :param gbn: GBNsend or GBNrecv object
 |      :param bufsize: number of pacekt cells in the buffer
 |  
 |  __setitem__(self, seq, item)
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  find_hole(self)
 |      Find the seq. of first hole in the buffer
 |      
 |      :return: seq if hole found
 |               next seq of the last packet, if no hole
 |  
 |  popfront(self)
 |      pop from front. Note: gbn.base will be modified
 |  
 |  --

### GBN receiving side   - TCP-like N 개 packet buffer 버전
N개까지의 packet들을 저장할 수 있는 packet buffer를 갖는 경우로 확장해 보자.

GBNrecv는 Rcv_DATA event를 받고 seqnum이 유효한 범위이면(즉, self.base <= seqnum < self.base + N),
받아서 저장한다. 아니면, ignore.
#### Saving DATA packets

In [10]:
N =8
DATA = 1
ACK = 2

class GBNrecv:
    def __init__(self):
        self.base = Seq(0)
        self.expected_seqnum = Seq(0)
        self.rcvpkt = PacketBuffer(self, N)
            
gbnrecv = GBNrecv()
gbnrecv.base = gbnrecv.expected_seqnum = Seq(253)

# Receiving DATA packets
data = b'Hello, world!!'
for seq in [Seq(253), Seq(0), Seq(1), Seq(254)]:
    pkt = make_pkt(seq, DATA, data)
    gbnrecv.rcvpkt[get_seqnum(pkt)] = pkt

print(gbnrecv.base)
print(gbnrecv.rcvpkt)

Seq(253)
[Seq(253), Seq(254), None, Seq(0), Seq(1), None, None, None]


#### Finding ACK number
생성해야 하는 ACK number는 receive buffer에 순서에 맞는 packet 번호의 다음 번호, 즉, 비어있는 packet 공간에 대한 seqnum로 ACK packet을 생성하여 feedback하면 된다. (Cummulative ACK임을 기억하자. 즉, 빈 공간 없이 순서에 맞게 도착했음을 알려야 한다.)

In [11]:
gbnrecv.expected_seqnum = gbnrecv.rcvpkt.find_hole()
print('ACK', int(gbnrecv.expected_seqnum))

ACK 255


#### Delivering data in order
Receive buffer에 저장된 packet 중에서 순서에 맞는(중간에 hole이 없는) 것을 pop 하고, data를 deliver한다.

In [13]:
base = gbnrecv.base
for seq in srange(base, gbnrecv.expected_seqnum):
    pkt = gbnrecv.rcvpkt.popfront()   # pop and remove front. gbnrecv.base will increase
    data = extract(pkt)
    print('deliver:', data)
print(gbnrecv.base, gbnrecv.expected_seqnum)
print(gbnrecv.rcvpkt)

Seq(255) Seq(255)
[None, Seq(0), Seq(1), None, None, None, None, None]
